In [0]:
from pyspark.sql.functions import to_date, col

### Read from bronze (landing) layer

In [0]:
# Account for landing files from https
storage_account_name = "factoredatathon"
storage_account_key = "yDTqsi+AifQJPvC5r7L5iFFdmmj+fbxWr280etWbWMPXWij0yfmiuLJH3sZ91TI7SwmfR1SBD8L7+AStGVUo3Q=="
container_name = "bronze"

spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net",
    f"{storage_account_key}"
)

In [0]:
# Read from bronze
file_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/*.CSV"
df = spark.read.option("header", "false").option("delimiter", "\t").csv(file_path)

# Adding column names
columns = [
    "GLOBALEVENTID", "SQLDATE", "MonthYear", "Year", "FractionDate",
    "Actor1Code", "Actor1Name", "Actor1CountryCode", "Actor1KnownGroupCode",
    "Actor1EthnicCode", "Actor1Religion1Code", "Actor1Religion2Code",
    "Actor1Type1Code", "Actor1Type2Code", "Actor1Type3Code", "Actor2Code",
    "Actor2Name", "Actor2CountryCode", "Actor2KnownGroupCode",
    "Actor2EthnicCode", "Actor2Religion1Code", "Actor2Religion2Code",
    "Actor2Type1Code", "Actor2Type2Code", "Actor2Type3Code", "IsRootEvent",
    "EventCode", "EventBaseCode", "EventRootCode", "QuadClass",
    "GoldsteinScale", "NumMentions", "NumSources", "NumArticles", "AvgTone",
    "Actor1Geo_Type", "Actor1Geo_FullName", "Actor1Geo_CountryCode",
    "Actor1Geo_ADM1Code", "Actor1Geo_Lat", "Actor1Geo_Long",
    "Actor1Geo_FeatureID", "Actor2Geo_Type", "Actor2Geo_FullName",
    "Actor2Geo_CountryCode", "Actor2Geo_ADM1Code", "Actor2Geo_Lat",
    "Actor2Geo_Long", "Actor2Geo_FeatureID", "ActionGeo_Type",
    "ActionGeo_FullName", "ActionGeo_CountryCode", "ActionGeo_ADM1Code",
    "ActionGeo_Lat", "ActionGeo_Long", "ActionGeo_FeatureID", "DATEADDED",
    "SOURCEURL"
]

df = df.toDF(*columns)

### Here we write into silver layer

In [0]:
# Convert 'YYYYMMDD' string to a date format
df = df.withColumn("DATE", to_date(col("SQLDATE"), "yyyyMMdd"))
# Filter rows where the date is greater than '2023-08-13'
df = df.filter(col("DATE") > '2023-08-13')

# Define the path where you want to save the Delta file in DBFS
delta_path = "/mnt/silver/eventsSilver"
# Write the DataFrame as a Delta file
df = df.repartition(32)
df.write.format("delta").mode("overwrite").save(delta_path)